# **Click on Runtime and select Change runtime type option. And then choose GPU from the Hardware Accelerator.**


# **Mount your Google drive **

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# **List the files on your google drive**

In [0]:
! ls '/content/drive/My Drive'

# Change the directory of your **drive**

In [0]:
import os
os.chdir('/content/drive/My Drive/HW6/hw6_fcn')


# **Import statements**

In [0]:
%tensorflow_version 1.x
import tensorflow as tf
import os.path
import warnings
import glob

import data_utils

TensorFlow 1.x selected.


# Function to load the pretrained model provided    (10 points)

In [0]:
def load_model(sess, model_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    sess: Tensorflow session
    model_path: Path to the vgg folder, it was provided to you
    return: return tuple of tensors (input_image, keep_prob, out_layer3, out_layer4, out_layer7)
    """
    # TODO: Implement function
    # to load the model and weights use tf.saved_model.loader.load
    # Use this tensor names to fetch the layers from pretrained vgg model:
    # input_tensor         :  'image_input:0'
    # kee_prob_tensor_name :  'keep_prob:0'
    # out_layer3_tensor    :  'layer3_out:0'
    # out_layer4_tensor    :  'layer4_out:0'
    # out_layer7_tensor    :  'layer7_out:0' 
    # TODO load the model
    # load the model and weights
    model = tf.saved_model.loader.load(sess, ['vgg16'], model_path)
    graph = tf.get_default_graph()
    input_tensor         =   graph.get_tensor_by_name('image_input:0')
    keep_prob_tensor     =   graph.get_tensor_by_name('keep_prob:0')
    out_layer3_tensor    =   graph.get_tensor_by_name('layer3_out:0')
    out_layer4_tensor    =   graph.get_tensor_by_name('layer4_out:0')
    out_layer7_tensor    =   graph.get_tensor_by_name('layer7_out:0')
    
    return input_tensor, keep_prob_tensor, out_layer3_tensor, out_layer4_tensor, out_layer7_tensor

# Create FCN-8 architecture                    (10 points)

In [0]:
def create_layers(layer3_out, layer4_out, layer7_out, num_classes):
    """
    Pretrained model provided is VGG
    Build a FCN model. Please read the paper Fully Convolutional Networks to complete this task.
    layer3_out : Output Tensor of Layer 3 VGG model
    layer4_out : Output Tensor of Layer 4 VGG model
    layer7_out : Output Tensor of Layer 7 VGG model
    return: Tensor for the last layer of output
    """
    # TODO: Implement function
    layer3, layer4, layer7 = layer3_out, layer4_out, layer7_out
    fcn8 = tf.layers.conv2d(layer7, filters=num_classes, kernel_size=1, name="fcn8")
    fcn9 = tf.layers.conv2d_transpose(fcn8, filters=layer4.get_shape().as_list()[-1],
    kernel_size=4, strides=(2, 2), padding='SAME', name="fcn9")
    fcn9_skip_connected = tf.add(fcn9, layer4, name="fcn9_plus_vgg_layer4")
    fcn10 = tf.layers.conv2d_transpose(fcn9_skip_connected, filters=layer3.get_shape().as_list()[-1],
    kernel_size=4, strides=(2, 2), padding='SAME', name="fcn10_conv2d")
    fcn10_skip_connected = tf.add(fcn10, layer3, name="fcn10_plus_vgg_layer3")
    fcn11 = tf.layers.conv2d_transpose(fcn10_skip_connected, filters=num_classes,
    kernel_size=16, strides=(8, 8), padding='SAME', name="fcn11")

    return fcn11


# Define loss function and optimizer             (15 points)

In [0]:
def optimizer(network_last_layer, actual_label, learning_rate, n_classes):
    """
    In this function create ops for tensorflow loss and optimizer
    Params:
    network_last_layer: Tensorflow tensor for the last layer in the neural network
    actual_label: Tensorflow placeholder for the actual label of the image
    learning_rate: Tensroflow placeholder for the learning rate
    n_classes: Number of classes
    return: Tuple of (logits, training_op, crossentropy_loss)
    """
    # TODO: Implement function

    logits = tf.reshape(network_last_layer, (-1, n_classes), name="fcn_logits")
    correct_label_reshaped = tf.reshape(actual_label, (-1, n_classes))
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=correct_label_reshaped[:])
    ce_loss = tf.reduce_mean(cross_entropy, name="fcn_loss")
    train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(ce_loss, name="fcn_train_op")

    return logits, train_op, ce_loss



# Method to perform training             (25 points)

In [0]:
def run_fcn():
    tf.reset_default_graph()
    image_shape = (160, 576)
    n_classes = 2
    #plc b_n img0, img1, nclasses
    correct_label = tf.placeholder(tf.int32, [1, image_shape[0], image_shape[1], n_classes])
    learning_rate = tf.placeholder(tf.float32)
    keep_prob = tf.placeholder(tf.float32)
    
    
    data_dir = '/content/drive/My Drive/HW6/hw6_fcn/hw6_fcn/data/'
    runs_dir = './runs'
    NUM_EPOCHS = 30 # initialize number of epochs # We used 20 to get the results provided in the document.
    BATCH_SIZE  = 1 # initialize with batch size for training # We used 1 to get the results provided in the document.
    #Get training data iterator 
    iterator = data_utils.get_training_iterator(os.path.join(data_dir, 'data/training'), image_shape, NUM_EPOCHS, BATCH_SIZE)
    imagebx_labelbx = iterator.get_next()
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    #training_dir = "/content/drive/My Drive/HW6/hw6_fcn/hw6_fcn/data/data/training"
    max_value = tf.placeholder(tf.int64, shape=[]) 
    with tf.Session(config=config) as sess:
        # Path to vgg model
        vggmodel_path = '/content/drive/My Drive/HW6/hw6_fcn/hw6_fcn/data/'
        # TODO: Build the Fully Convolutional Network using load_model, create_layers, and optimizer function
        image_input, keep_prob, layer3, layer4, layer7 = load_model(sess, vggmodel_path)
        model_output = create_layers(layer3, layer4, layer7, n_classes)
        logits, train_op, cross_entropy_loss = optimizer(model_output,correct_label ,learning_rate, n_classes)
        # TODO: Train the FCN network by iterating the tf.data iterator. Check the examples provided in the document 
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        keep_prob_value = 0.8
        learning_rate_value = 0.0001
        

        sess.run(iterator.initializer)
        for epoch in range(NUM_EPOCHS):
          print("EPOCH {} ...".format(epoch + 1))
          loss_t = 0
          for i in range(289): 
            #print(i)
            value = sess.run(imagebx_labelbx) 
            X_batch = value[0]
            gt_batch = value[1]
            
            loss, _ = sess.run([cross_entropy_loss, train_op],
            feed_dict={image_input: X_batch, correct_label: gt_batch,
            keep_prob: keep_prob_value, learning_rate:learning_rate_value})
            loss_t += loss;
          print("Loss = {:.3f}".format(loss_t))


        print("Loss = {:.3f}".format(loss_t))
        print()
    #TODO: Once training is completed, use the trained model on test dataset.
        data_utils.test_images(runs_dir, data_dir, sess, logits,  keep_prob, image_input, image_shape)

# After completing the code in above cells, call run_fcn() method.

In [0]:
if __name__ == "__main__":
    run_fcn()